In [ ]:
import pandas as pd
import random
import json
import numpy as np
import sys
import requests
import xmltodict
from PIL import Image, ImageDraw
import json
import matplotlib.pyplot as plt
import torch.optim as optim
from math import *
import shutil
import copy
from torch.utils.checkpoint import checkpoint 
import time
from log import *
from torch.utils.data import Dataset
import xml.etree.ElementTree as et
from torchvision import datasets,transforms,utils
from torch.utils.data import DataLoader

import torch
import torch.nn as nn

from torchvision.models import * 
from torch.optim import lr_scheduler

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
epoch=20
file_name="test"
if not os.path.isdir("./ensemble_result"):
    os.mkdir("./ensemble_result")
Path="./ensemble_result/"+file_name+"/"
log_path=Path+"Process_log.txt"
batch_size=256

lr=0.005                  
step_size=10
gamma=0.1


if os.path.isdir(Path):
    print("이미 폴더가 존재합니다.")
    sys.exit()
os.mkdir(Path)

print("log_path :",log_path)
TXT=log(log_path)
TXT.open()

In [ ]:
# rotation 필요, 단일 데이터 라벨링 이기 때문에 다중 데이터 같은 경우 bounding box 에서 에러 날 수 있으니 확인

def Data_Pretreatment(path="./166.약품식별 인공지능 개발을 위한 경구약제 이미지 데이터/01.데이터/1.Training/",
                      category=["object_name","RHF","RVF","RG","path"],
                      img_move_path="./DATA/",
                      img_move=0,
                      data_augmentation={"RHF":1,"RVF":1,"RG":1}):
    
    if os.path.isfile("./dataframe.csv"):
        DATAFRAME=pd.read_csv("./dataframe.csv")
        yshape=DATAFRAME.shape[0]
        object_type=[i for i in range(DATAFRAME["object_name"].iloc[yshape-1]+1)]
        return pd.read_csv("./dataframe.csv"),object_type
    
    print("data pretreatment")

    #transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(p=0.5),
    #transforms.RandomGrayscale(p=0.75),

    label_path=path+"라벨링데이터/"
    image_path=path+"원천데이터/"
    if img_move==1:
        if not os.path.isdir("./DATA"):
            os.mkdir("./DATA")

    dataframe=np.array([])
    dataframe = np.empty((0,len(category)), int)
    
    img_draw_stack=0
    object_type=[]
    object_type_idx=0


    for big_category in os.listdir(image_path):
          
        big_category_path=image_path+big_category+"/"

        for mid_category in os.listdir(big_category_path):

            if len(mid_category.split(".zip"))>=2:continue
            mid_category_path=big_category_path+mid_category+"/"
            
            for object_name in os.listdir(mid_category_path):

                #print("object name :",object_name)

                object_path=mid_category_path+object_name+"/"
                #print(os.listdir(object_path))

                for file_name in os.listdir(object_path):

                    img2json_path=file_name.split(".")[0]+".json"
                    mid_category_replace=mid_category.replace("S","L")
                    object_label_path=label_path+big_category+"/"+mid_category_replace+"/"+object_name+"_json"+"/"+img2json_path
                    with open (object_label_path, "r",encoding='UTF-8') as f:
                        data = json.load(f)

                    NAME=data['images'][0]['dl_name'].replace("/","-")
                    object_file_path=img_move_path+NAME+"/"

                    

                

        
                    if not NAME in object_type:
                        object_type.append(NAME)
                        object_type_idx+=1

                    """
                    if img_draw_stack==0:
                        img_draw_stack=1


                        img = Image.open(object_file_path+file_name).convert('RGB')
                        Size=[data['images'][0]['width'],data['images'][0]['height']]
                        img=img.resize((256,256))
                        img.show()
                        BOX=data['annotations'][0]['bbox']
                        alpha=[256/Size[0],256/Size[1]]
                        BOX=[BOX[i]*alpha[i%2]for i in range(4)]
                        draw = ImageDraw.Draw(img)
                        draw.rectangle((BOX[0],BOX[1],BOX[0]+BOX[2],BOX[1]+BOX[3]), outline=(0,255,0), width = 3)

                        img.show()
                        sys.exit()
                    
                    """

                    Size=[data['images'][0]['width'],data['images'][0]['height']]
                    BOX=data['annotations'][0]['bbox']

                    if img_move==1:
                        img_size=(480,480)
                        img = Image.open(object_path+file_name).convert('RGB')
                        Size=img.size
                        if not os.path.isdir(object_file_path):
                            os.mkdir(object_file_path)
                        center=[(2*BOX[0]+BOX[2])/2,(2*BOX[1]+BOX[3])/2]
                        img=img.crop((center[0]-240,center[1]-240,center[0]+240,center[1]+240))
                        img=img.resize((240,240))
                        img.save(object_file_path+file_name,"PNG")
                        #shutil.copyfile(object_path+file_name, object_file_path+file_name)
                    dataframe=np.append(dataframe,np.array([[object_type_idx-1,0,0,0,object_file_path+file_name]]),axis=0)
                    if data_augmentation["RHF"]:
                        dataframe=np.append(dataframe,np.array([[object_type_idx-1,1,0,0,object_file_path+file_name]]),axis=0)
                    if data_augmentation["RVF"]:
                        dataframe=np.append(dataframe,np.array([[object_type_idx-1,0,1,0,object_file_path+file_name]]),axis=0)
                    if data_augmentation["RG"]:
                        dataframe=np.append(dataframe,np.array([[object_type_idx-1,0,0,1,object_file_path+file_name]]),axis=0)
                    #print(NAME,BOX)

    DATAFRAME=pd.DataFrame(data=dataframe,columns=category)
    #print("DATAFRAME_TYPE :",type(DATAFRAME)
    # )

    return DATAFRAME,object_type

    

In [ ]:
def DataFrame_dividing(DataFrame,category=["object_name","path"]):

    train=np.array([])
    train = np.empty((0,len(category)), int)
    test=np.array([])
    test = np.empty((0,len(category)), int)

    #print(DataFrame)
    

    class_stack=[]
    last_obj=-1

    #print(DataFrame.shape)
    #print(DATAFRAME)

    for i in range(DataFrame.shape[0]):

        Ip=[]
        for g in range(len(category)):Ip.append(DataFrame.iloc[i][category[g]])
        if last_obj==-1:
            last_obj=Ip[0]

        #print(last_obj,Ip[0])

        if last_obj!=Ip[0]:
            #print("##")
            T=int(len(class_stack)/10 * 7)
            #print("TT",T)
            for g in range(T):
                train=np.append(train,np.array([class_stack[g]]),axis=0)
                #print(train,np.array([class_stack[g]]),class_stack[g])
                #np.append(dataframe,np.array([[object_type_idx-1, BOX[0], BOX[1], BOX[0]+BOX[2], BOX[1]+BOX[3], object_file_path+file_name]]),axis=0)
            for g in range(T,len(class_stack)):
                test=np.append(test,np.array([class_stack[g]]),axis=0)

            class_stack=[]
            last_obj=Ip[0]
        class_stack.append(Ip)

    if class_stack!=[]:
        T=int(len(class_stack)/10 * 7)
        for g in range(T):
            train=np.append(train,np.array([class_stack[g]]),axis=0)
            #np.append(dataframe,np.array([[object_type_idx-1, BOX[0], BOX[1], BOX[0]+BOX[2], BOX[1]+BOX[3], object_file_path+file_name]]),axis=0)
        for g in range(T,len(class_stack)):
            test=np.append(test,np.array([class_stack[g]]),axis=0)
        
    train=pd.DataFrame(data=train,columns=category)
    test=pd.DataFrame(data=test,columns=category)

    return train,test



    

In [ ]:
def TransForm(size=256):
    size=256
    train_transform = transforms.Compose([
        #transforms.Resize(size),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        #transforms.RandomVerticalFlip(),
        #transforms.RandomGrayscale(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
    ])

    test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
    ])
    return train_transform, test_transform

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,labels,transforms):
        self.labels=labels
        #print(self.labels)
        self.transforms=transforms

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_path=self.labels["path"].iloc[idx]
        label=self.labels["object_name"].iloc[idx]
        #print(idx,self.labels.iloc[idx])

        img=Image.open(img_path)

        if self.labels["RHF"].iloc[idx]:
            hflipper = transforms.RandomHorizontalFlip(p=0.5)
            img=hflipper(img)

        elif self.labels["RVF"].iloc[idx]:
            vflipper = transforms.RandomVerticalFlip(p=0.5)
            img=vflipper(img)

        elif self.labels["RG"].iloc[idx]:
            scaler = transforms.RandomGrayscale(p=0.75)
            img=scaler(img)

        if self.transforms:
            img=self.transforms(img)

       # OUTPUT={"iamge":img,"label":label}

        return img,label

In [ ]:
# rotation 필요, 단일 데이터 라벨링 이기 때문에 다중 데이터 같은 경우 bounding box 에서 에러 날 수 있으니 확인
#image folder produce data file
def if_pd_datafile(path="./166.약품식별 인공지능 개발을 위한 경구약제 이미지 데이터/01.데이터/1.Training/",
                      category=["object_name","RHF","RVF","RG","path"],
                      img_move_path="./DATA/",
                      data_augmentation={"RHF":1,"RVF":1,"RG":1}):
    

    label_path=path+"라벨링데이터/"
    image_path=path+"원천데이터/"
    if not os.path.isdir("./DATA"):
         os.mkdir("./DATA")
    if not os.path.isdir("./DATA/train"):
         os.mkdir("./DATA/train")
    if not os.path.isdir("./DATA/test"):
         os.mkdir("./DATA/test")

    class_stack=[]
    
    object_type=[]
    object_type_idx=0


    for big_category in os.listdir(image_path):
          
        big_category_path=image_path+big_category+"/"

        for mid_category in os.listdir(big_category_path):

            if len(mid_category.split(".zip"))>=2:continue
            mid_category_path=big_category_path+mid_category+"/"
            
            for object_name in os.listdir(mid_category_path):

                #print("object name :",object_name)

                object_path=mid_category_path+object_name+"/"
                #print(os.listdir(object_path))

                for file_name in os.listdir(object_path):

                    img2json_path=file_name.split(".")[0]+".json"
                    mid_category_replace=mid_category.replace("S","L")
                    object_label_path=label_path+big_category+"/"+mid_category_replace+"/"+object_name+"_json"+"/"+img2json_path
                    with open (object_label_path, "r",encoding='UTF-8') as f:
                        data = json.load(f)

                    NAME=data['images'][0]['dl_name'].replace("/","-")
        
                    if not NAME in object_type:
                        if object_type!=[]:
                            T=int(len(class_stack)/10 * 7)
                            #train
                            for g in range(T):
                                DATA=class_stack[g]
                                BOX=DATA[4]

                                img = Image.open(DATA[3]).convert('RGB')
                                object_file_path=img_move_path+"train/"+DATA[2]+"/"
                                if not os.path.isdir(object_file_path):
                                    os.mkdir(object_file_path)

                                center=[(2*BOX[0]+BOX[2])/2,(2*BOX[1]+BOX[3])/2]
                                #img=img.crop((center[0]-240,center[1]-240,center[0]+240,center[1]+240))
                                img=img.crop((BOX[0],BOX[1],BOX[0]+BOX[2],BOX[1]+BOX[3]))
                                #print(BOX)
                                #print(img.size)
                                img=img.resize((240,240))

                                if DATA[1]["RHF"]:
                                    hflipper = transforms.RandomHorizontalFlip(p=0.5)
                                    img=hflipper(img)

                                elif DATA[1]["RVF"]:
                                    vflipper = transforms.RandomVerticalFlip(p=0.5)
                                    img=vflipper(img)

                                elif DATA[1]["RG"]:
                                    scaler = transforms.RandomGrayscale(p=0.75)
                                    img=scaler(img)

                                img.save(object_file_path+DATA[5],"PNG")


                            #test
                            for g in range(T,len(class_stack)):
                                DATA=class_stack[g]
                                BOX=DATA[4]

                                img = Image.open(DATA[3]).convert('RGB')
                                object_file_path=img_move_path+"test/"+DATA[2]+"/"
                                if not os.path.isdir(object_file_path):
                                    os.mkdir(object_file_path)

                                center=[(2*BOX[0]+BOX[2])/2,(2*BOX[1]+BOX[3])/2]
                                img=img.crop((BOX[0],BOX[1],BOX[0]+BOX[2],BOX[1]+BOX[3]))
                                img=img.resize((240,240))

                                if DATA[1]["RHF"]:
                                    hflipper = transforms.RandomHorizontalFlip(p=0.5)
                                    img=hflipper(img)

                                elif DATA[1]["RVF"]:
                                    vflipper = transforms.RandomVerticalFlip(p=0.5)
                                    img=vflipper(img)

                                elif DATA[1]["RG"]:
                                    scaler = transforms.RandomGrayscale(p=0.75)
                                    img=scaler(img)

                                img.save(object_file_path+DATA[5],"PNG")

                        object_type.append(NAME)
                        class_stack=[]
                        object_type_idx+=1

                    
                    class_stack.append([object_type_idx,data_augmentation,NAME,object_path+file_name,data['annotations'][0]['bbox'],file_name])
                    
                
                    #print(NAME,BOX)
    if class_stack!=[]:
        T=int(len(class_stack)/10 * 7)
        #train
        for g in range(T):
            DATA=class_stack[g]
            BOX=DATA[4]

            img = Image.open(DATA[3]).convert('RGB')
            object_file_path=img_move_path+"train/"+DATA[2]+"/"
            if not os.path.isdir(object_file_path):
                os.mkdir(object_file_path)

            center=[(2*BOX[0]+BOX[2])/2,(2*BOX[1]+BOX[3])/2]
            img=img.crop((BOX[0],BOX[1],BOX[0]+BOX[2],BOX[1]+BOX[3]))

            img=img.resize((240,240))
            if DATA[1]["RHF"]:
                hflipper = transforms.RandomHorizontalFlip(p=0.5)
                img=hflipper(img)

            elif DATA[1]["RVF"]:
                vflipper = transforms.RandomVerticalFlip(p=0.5)
                img=vflipper(img)

            elif DATA[1]["RG"]:
                scaler = transforms.RandomGrayscale(p=0.75)
                img=scaler(img)

            img.save(object_file_path+DATA[5],"PNG")


        #test
        for g in range(T,len(class_stack)):
            DATA=class_stack[g]
            BOX=DATA[4]

            img = Image.open(DATA[3]).convert('RGB')
            object_file_path=img_move_path+"test/"+DATA[2]+"/"
            if not os.path.isdir(object_file_path):
                os.mkdir(object_file_path)

            center=[(2*BOX[0]+BOX[2])/2,(2*BOX[1]+BOX[3])/2]
            img=img.crop((BOX[0],BOX[1],BOX[0]+BOX[2],BOX[1]+BOX[3]))
            img=img.resize((240,240))

            if DATA[1]["RHF"]:
                hflipper = transforms.RandomHorizontalFlip(p=0.5)
                img=hflipper(img)

            elif DATA[1]["RVF"]:
                vflipper = transforms.RandomVerticalFlip(p=0.5)
                img=vflipper(img)

            elif DATA[1]["RG"]:
                scaler = transforms.RandomGrayscale(p=0.75)
                img=scaler(img)

            img.save(object_file_path+DATA[5],"PNG")
            

    return object_type

    #DATAFRAME=pd.DataFrame(data=dataframe,columns=category)
    #print("DATAFRAME_TYPE :",type(DATAFRAME)
    # )

    #return DATAFRAME,object_type

    

In [ ]:
def produce_Dataset(train_dataset,test_dataset,train_transform,test_transform,batch_size=64):

  #[train,train_transform]
  #print("train")
  train_dataset = CustomDataset(train_dataset,train_transform)
  #print("test")
  test_dataset = CustomDataset(test_dataset,test_transform)

  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

  test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

  return {"dataset":train_dataset,"dataloader":train_dataloader},{"dataset":test_dataset,"dataloader":test_dataloader}


In [ ]:
def IF_produce_Dataset(train_transform,test_transform,batch_size=64):

  train_dataset = datasets.ImageFolder(os.path.join("./DATA/train"),train_transform)

  test_dataset = datasets.ImageFolder(os.path.join("./DATA/test"),test_transform)

  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

  test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

  return {"dataset":train_dataset,"dataloader":train_dataloader},{"dataset":test_dataset,"dataloader":test_dataloader}


In [ ]:
def txt_print(input):
    TXT.print_log(input)
    

In [ ]:
def test_model(TEST,PATH):

    """
    model_name=[['resnet50','resnet50_v2.pth'],['resnet101','resnet101_v1.pth'],['resnet101','resnet101_v2.pth'],['resnet152','resnet152_v2.pth']]
    model_weights=['ResNet50_Weights.IMAGENET1K_V2','ResNet101_Weights.IMAGENET1K_V1','ResNet101_Weights.IMAGENET1K_V2','ResNet152_Weights.IMAGENET1K_V2']
    """

    #sys.stdout=open(PATH+"train_result.txt","w")
    since = time.time()



    sum_running_loss = 0.0
    sum_running_corrects = 0.0 #[0]*(len(MPI.object_stack)+2)

    max_running_loss = 0.0
    max_running_corrects = 0.0

    # 데이터를 반복
    for i,batch in enumerate(TEST["learning_dataloader"]):
        #print("idx",i)

        inputs,labels=batch
        inputs = inputs.to(device)
        #labels=torch.tensor(labels)
        labels = torch.LongTensor.clone(labels).to(device)

        sum_loss=0.0
        max_loss=0.0
        sum_stack=[]
        max_stack=[]
        for idx in range(len(TEST['model_name'])):
            print(idx)

            if TEST["model_name"][idx][0]=='resnet50':
                model_conv = resnet50(weights=TEST['model_weights'][0])#(weights="ResNet152_Weights.IMAGENET1K_V1")
            elif TEST["model_name"][idx][0]=='resnet101_v1':
                model_conv = resnet101(weights=TEST['model_weights'][1])
            elif TEST["model_name"][idx][0]=='resnet101_v2':
                model_conv = resnet101(weights=TEST['model_weights'][2])
            elif TEST["model_name"][idx][0]=='resnet152':
                model_conv = resnet152(weights=TEST['model_weights'][3])

            for param in model_conv.parameters():param.requires_grad = False
            num_ftrs = model_conv.fc.in_features
            model_conv.fc = nn.Linear(num_ftrs, len(TEST['object_type']))
            model_conv = model_conv.to(device)
            print(TEST["model_name"][idx][0],PATH+TEST['model_name'][idx][1])
            TL=torch.load(PATH+TEST['model_name'][idx][1])
            model_conv.load_state_dict(TL)
            model_conv.eval()
            #print(idx)

            outputs = model_conv(inputs)
            #print(torch.min(outputs))
            outputs+=abs(torch.min(outputs).item())+0.01
            #print(torch.min(outputs))
            _, preds = torch.max(outputs, 1)

            max_stack.append(preds.detach().cpu().numpy().tolist())
            sum_stack.append(outputs)
            AA=TEST["criterion"](outputs, labels)
            max_loss += AA
            sum_loss += AA

        max_stack=max_stack[0]
        #print("stack",stack)
        #print(len(stack))

        max_loss/=len(TEST['model_name'])
        sum_loss/=len(TEST['model_name'])
        ANS=[[0 for i in range(7)]for g in range(len(max_stack))]
        #print("ans",ANS)
        #print(len(ANS))
        Su=sum_stack[0]
        #print(stack)
        #print(0,Su)
        for g in range(len(max_stack)):
            ANS[g][int(max_stack[g])]+=1

        for i in range(1,len(sum_stack)):
            Su+=sum_stack[i]
            #print(i,Su)
        max_preds=torch.Tensor([ANS[i].index(max(ANS[i]))for i in range(len(ANS))]).cuda()
        _, sum_preds = torch.max(Su, 1)
        #print("preds",preds)
        #print(len(preds))

        #print("finish",Su)

        # 통계
        sum_running_loss += sum_loss.item() * inputs.size(0)
        sum_running_corrects += torch.sum(sum_preds == labels.data)

        max_running_loss += max_loss.item() * inputs.size(0)
        max_running_corrects += torch.sum(max_preds == labels.data)

    
    TXT.close()
    TXT.open()

    time_elapsed = time.time() - since

    TXT.print_log(f'ensemble Testing complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    TXT.print_log("")
    TXT.print_log(f'max test Acc: {max_running_corrects.double() / TEST["learning_dataset_size"]}')
    TXT.print_log(f'max test Acc\'s loss : {max_running_loss / TEST["learning_dataset_size"]}')
    TXT.print_log(f'sum test Acc: {sum_running_corrects.double() / TEST["learning_dataset_size"]}')
    TXT.print_log(f'sum test Acc\'s loss : {sum_running_loss / TEST["learning_dataset_size"]}')

    TXT.close()



In [ ]:


TXT.print_log("Hyperparameter setting")
TXT.print_log("----------------------")



#Hyperparameter_path=Path+"Hyperparameter.txt"
#sys.stdout=open(Hyperparameter_path,"w")

#---------------------Dataset setting---------------------#

category=["object_name","RHF","RVF","RG","path"]
#object_type=if_pd_datafile(category=category)
object_type=[0]*5
train_transform,test_transform=TransForm(240)
"""
DATAFRAME,object_type=Data_Pretreatment(category=category,img_move=0)
train,test=DataFrame_dividing(DATAFRAME,category)

#print("test")
#print(test)

Train,Test=produce_Dataset(train,test,train_transform,test_transform,batch_size)
"""
Train,Test=IF_produce_Dataset(train_transform,test_transform,batch_size)

learning_dataset={"train":Train["dataset"],"test":Test["dataset"]}
learning_dataloader={"train":Train["dataloader"],"test":Test["dataloader"]}
learning_dataset_size={"train":len(Train["dataset"]),"test":len(Test["dataset"])}

#-----
# ---------------Dataset setting---------------------#

"""
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

    
dataiter = iter(learning_dataloader["train"])
images, labels = next(dataiter)

# 이미지 보여주기
imshow(utils.make_grid(images))
# 정답(label) 출력
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))
"""

#---------------------model---------------------#
model_name=[['resnet50','resnet50_v2.pth']]#,['resnet101_v1','resnet101_v1.pth'],['resnet101','resnet101_v2.pth']]#,['resnet152','resnet152_v2.pth']]
#'resnet50_v1.pth','resnet152_v1.pth'
model_weights=['ResNet50_Weights.IMAGENET1K_V2','ResNet101_Weights.IMAGENET1K_V1','ResNet101_Weights.IMAGENET1K_V2','ResNet152_Weights.IMAGENET1K_V2']
weights=[1,1,1]
#resnet
#num_ftrs = model_conv.fc.in_features
#model_conv.fc = nn.Linear(num_ftrs, len(object_type))


#densenet
#num_ftrs = model_conv.fc.in_features
#model_conv.classifier = nn.Linear(num_ftrs, len(object_type))

#efficientnet
#num_ftrs = model_conv.classifier[1].in_features
#model_conv.classifier[1] = nn.Linear(num_ftrs, len(object_type))

#model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()
    #TXT.print_log(str("Loss Function is CrossEntropyLoss"))



#---------------------model---------------------#
#TXT.print_log("---------------------------")

In [ ]:
#train_model(model, criterion, optimizer, scheduler, learning_dataloader,learning_dataset_size,EPOCH,PATH)
TXT.print_log("train & test log")
TXT.print_log("---------")
TEST={
       "criterion":criterion,
        "learning_dataloader":learning_dataloader["test"],
        "learning_dataset_size":learning_dataset_size["test"],
        "model_name":model_name,
        "model_weights":model_weights,
        "object_type":object_type,
        "weights":weights
}

test_model(TEST,'./ensemble_file/')
TXT.open()
TXT.print_log("---------------------------")
TXT.close()



In [ ]:
learning_dataset_size["test"]